# Amazon SageMaker Multi-Model Endpoints using PyTorch

> *This notebook works well with SageMaker Studio kernel `Python 3 (Data Science)`, or SageMaker Notebook Instance kernel `conda_python3`*

[Amazon SageMaker multi-model endpoints](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html)를 이용해서, 고객들은 수천개의 모델에 대한 endpoint 을 생성할 수 있습니다. <br/>
이러한 endpoint 는 공통된 추론 컨테이너에서 제공할 수 있는 많은 모델 중 하나가 요청시 호출되어야 하고 드물게 호출되는 모델이 추가 대기 시간을 발생시키는 것이 허용되는 사용 사례에 매우 적합합니다. <br/>
지속적으로 낮은 추론 지연 시간이 필요한 애플리케이션의 경우 전통적인 엔드포인트가 여전히 최선의 선택입니다.<br/>
지연시간이 견딜수 있는 수준이고 비용 최적화가 보다 중요한 경우, A/B/n 테스트를 위해 MME 를 사용할 수 있습니다.

상용 환경에서 SageMaker를 사용하여 A/B 테스트 수행에 대한 자세한 내용은 아래를 참고하시기 바랍니다. <br/>
https://aws.amazon.com/blogs/machine-learning/a-b-testing-ml-models-in-production-using-amazon-sagemaker/ 

Multi-model endpoints 를 생성하고 사용하는 법을 보여드리기 위해 이 노트북은 [SageMaker PyTorch framework container](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html) 으로 traning 된 모델을 사용한 예제를 제공합니다. 

단순하게 진행하기 위해 두가지 모델을 training 하고 배포하는 단순한 A/B 시나리오로 시작하도록 하겠습니다. 

다른 MME 사용사례는 아래 링크들을 참조하실 수 있습니다. :

- 3가지로 세그먼트화된 집값 예측 모델 예제 ([Scikit-Learn framework](https://github.com/aws/amazon-sagemaker-examples/tree/master/advanced_functionality/multi_model_sklearn_home_value), [XGBoost pre-built algorithm](https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/multi_model_xgboost_home_value), [Linear Learner algorithm](https://github.com/aws/amazon-sagemaker-examples/tree/master/advanced_functionality/multi_model_linear_learner_home_value))

- MXNet을 사용하는 BYOC 예제 ([코드](https://github.com/aws/amazon-sagemaker-examples/tree/master/advanced_functionality/multi_model_bring_your_own), [관련문서](https://docs.aws.amazon.com/sagemaker/latest/dg/build-multi-model-build-container.html))

## Contents

1. [The example use case: MNIST](#The-example-use-case:-MNIST)
1. [Train multiple models](#Train-multiple-models)
1. [Check single-model deployment](#Check-single-model-deployment)
1. [Create the Multi-Model Endpoint with the SageMaker SDK](#Create-the-Multi-Model-Endpoint-with-the-SageMaker-SDK)
  1. [Deploy the Multi-Model Endpoint](#Deploy-the-Multi-Model-Endpoint)
  1. [Dynamically deploying models to the endpoint](#Dynamically-deploying-models-to-the-endpoint)
1. [Get predictions from the endpoint](#Get-predictions-from-the-endpoint)
1. [Updating a model](#Updating-a-model)
1. [Clean up](#Clean-up)


시작하기 전에 필요한 라이브러리를 로드하고 몇가지 설정을 정의할 것입니다. <br/>
데이터가 저장되는 S3 버킷은 편이에 따라 변경할 수 있습니다만 아래 예에서는 SageMaker default bucket 을 사용합니다. 

In [ ]:
# Python Built-Ins:
from datetime import datetime
import os
import json
import logging
from tempfile import TemporaryFile
import time

# External Dependencies:
import boto3
from botocore.exceptions import ClientError
import numpy as np
import sagemaker
from sagemaker.multidatamodel import MultiDataModel
from sagemaker.pytorch import PyTorch as PyTorchEstimator, PyTorchModel

smsess = sagemaker.Session()
role = sagemaker.get_execution_role()

# Configuration:
bucket_name = smsess.default_bucket()
prefix = "mnist/"
output_path = f"s3://{bucket_name}/{prefix[:-1]}"

## The example use case: MNIST

MNIST 는 손으로 씌여진 숫자 분류에 폭넓게 사용되는 데이터셋이며 70,000 개의 라벨링된 28x28 픽셀의 회색 이미지들로 구성되어 있습니다. 

데이터셋은 훈련용으로 60,000개, 테스트용으로 10,000개로 나뉩니다.

이 예에서는 MNIST 데이터를 public S3 버킷으로부터 다운로드받아 이것을 default SageMaker 버킷으로 업로드합니다. 

In [ ]:
def fetch_sample_data(
    to_bucket: str,
    to_prefix: str,
    from_bucket: str = "sagemaker-sample-files",
    from_prefix: str = "datasets/image/MNIST",
    dataset: str = "mnist-train",
):
    DATASETS = {
        "mnist-train": ["train-images-idx3-ubyte.gz", "train-labels-idx1-ubyte.gz"],
        "mnist-test": ["t10k-images-idx3-ubyte.gz", "t10k-labels-idx1-ubyte.gz"],
    }

    if dataset not in DATASETS:
        raise ValueError(f"dataset '{dataset}' not in known set: {set(DATASETS.keys())}")

    if len(from_prefix) and not from_prefix.endswith("/"):
        from_prefix += "/"
    if len(to_prefix) and not to_prefix.endswith("/"):
        to_prefix += "/"

    s3client = boto3.client("s3")
    for key in DATASETS[dataset]:
        # If you're in the same region as the source bucket, could consider copy_object() instead:
        with TemporaryFile() as ftmp:
            s3client.download_fileobj(from_bucket, f"{from_prefix}{key}", ftmp)
            ftmp.seek(0)
            s3client.upload_fileobj(ftmp, to_bucket, f"{to_prefix}{key}")


train_prefix = f"{prefix}data/train"
fetch_sample_data(to_bucket=bucket_name, to_prefix=train_prefix, dataset="mnist-train")
train_s3uri = f"s3://{bucket_name}/{train_prefix}"
print(f"Uploaded training data to {train_s3uri}")

test_prefix = f"{prefix}data/test"
fetch_sample_data(to_bucket=bucket_name, to_prefix=test_prefix, dataset="mnist-test")
test_s3uri = f"s3://{bucket_name}/{test_prefix}"
print(f"Uploaded training data to {test_s3uri}")

In [ ]:
print("Training data:")
!aws s3 ls --recursive $train_s3uri
print("Test data:")
!aws s3 ls --recursive $test_s3uri

## Train multiple models

다음 섹션에서는 같은 데이터셋에 대해 SageMaker PyTorch Framework Container 를 이용하여 다중 모델을 training 할 것입니다. 

단순하게 진행하기위해 같은 코드이지만 다른 hyperprarmeter 를 가진 `A` 와 `B` 두개의 모델을 만들 것입니다. 

In [ ]:
def get_estimator(base_job_name, hyperparam_overrides={}):
    hyperparameters = {
        "batch-size": 128,
        "epochs": 20,
        "learning-rate": 1e-3,
        "log-interval": 100,
    }
    for k, v in hyperparam_overrides.items():
        hyperparameters[k] = v

    return PyTorchEstimator(
        base_job_name=base_job_name,
        entry_point="train.py",
        source_dir="code",  # directory of your training script
        role=role,
        # At the time of writing, this example gives a deployment error in container v1.8.1 with
        # upgraded TorchServe: so specifically setting "1.8.0". But "1.7" and "1.6" should be fine.
        framework_version="1.8.0",
        py_version="py3",
        instance_type="ml.c4.xlarge",
        instance_count=1,
        output_path=output_path,
        hyperparameters=hyperparameters,
    )


estimatorA = get_estimator(base_job_name="mnist-a", hyperparam_overrides={"weight-decay": 1e-4})
estimatorB = get_estimator(base_job_name="mnist-b", hyperparam_overrides={"weight-decay": 1e-2})

기본적으로 [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/)'s [Estimator.fit()](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit) 메소드는 training 이 완료될때까지 대기하면서 진행상의 정보나 로그를 노트북으로 스트리밍합니다. 

이것이 지원되는 유일은 구성은 아닙니다. 예를 들어 `wait=False` 로 설정하여 비동기적으로 작업을 시작하거나, 이전에 시작한 작업에 대해 소급하여 `wait()`을 호출함으로써 선택적으로 로그를 pulling 할수도 있습니다. 

아래 섹션에서는 두 훈련 작업을 병렬로 시작하고 실행될 때 `B`에서 로그를 스트리밍한 다음 `A`가 아직 완료되지 않은 경우 완료될 때까지 기다립니다.

In [ ]:
estimatorA.fit({"training": train_s3uri, "testing": test_s3uri}, wait=False)
print("Started estimator A training in background (logs will not show)")

print("Training estimator B with logs:")
estimatorB.fit({"training": train_s3uri, "testing": test_s3uri})

print("\nWaiting for estimator A to complete:")
estimatorA.latest_training_job.wait(logs=False)

## Check single-model deployment

multi-model 배포를 설정하기 전에 단일 모델이 잘 배포되고 기대한대로 동작하는지 확인하는 것이 도움이 될 수 있습니다.

In [ ]:
modelA = estimatorA.create_model(role=role, source_dir="code", entry_point="inference.py")

In [ ]:
predictorA = modelA.deploy(
    initial_instance_count=1,
    instance_type="ml.c5.xlarge",
)
predictorA.serializer = sagemaker.serializers.JSONSerializer()
predictorA.deserializer = sagemaker.deserializers.JSONDeserializer()

In [ ]:
def get_dummy_request():
    """Create a dummy predictor.predict example data (16 images of random pixels)"""
    return {"inputs": np.random.rand(16, 1, 28, 28).tolist()}


dummy_data = get_dummy_request()

start_time = time.time()
predicted_value = predictorA.predict(dummy_data)
duration = time.time() - start_time

print(f"Model took {int(duration * 1000):,d} ms")
np.array(predicted_value)[0]


잘 동작하는 것으로 보이므로 이 endpoint 는 더이상 필요가 없습니다. 

In [ ]:
predictorA.delete_endpoint(delete_endpoint_config=True)

## Create the Multi-Model Endpoint with the SageMaker SDK

### Create a SageMaker Model from one of the Estimators

multi-model endpoints 는 모델을 필요할 때 공유 컨테이너로 로드합니다. 그래서 runtime 을 정의하기 위해 estimator 중의 하나로부터 모델을 하나 생성해야 합니다. 

In [ ]:
model = estimatorA.create_model(role=role, source_dir="code", entry_point="inference.py")

### Create the Amazon SageMaker MultiDataModel entity

[```MultiDataModel```](https://sagemaker.readthedocs.io/en/stable/api/inference/multi_data_model.html) class 를 사용해서 multi-model endpoint 를 생성합니다. 

`sagemaker.model.Model` object 를 직접 전달하여 MultiDataModel 을 생성할 수 있습니다. <br/>
이 경우 Endpoint는 MultiDataModel이 배포되면 사용할 이미지와 환경 변수, 네트워크 격리 등에 대한 정보를 상속하게 됩니다.

참고로 [```MultiDataModel```](https://sagemaker.readthedocs.io/en/stable/api/inference/multi_data_model.html) 은 명시적으로 `sagemaker.model.Model` object 를 전달하지 않고도 생성할 수 있습니다. 


In [ ]:
# This is where our MME will read models from on S3.
multi_model_prefix = f"{prefix}multi-model/"
multi_model_s3uri = f"s3://{bucket_name}/{multi_model_prefix}"
print(multi_model_s3uri)

In [ ]:
mme = MultiDataModel(
    name="mnist-multi-" + datetime.now().strftime("%Y-%m-%d-%H-%M-%S"),
    model_data_prefix=multi_model_s3uri,
    model=model,  # passing our model
    sagemaker_session=smsess,
)

### Deploy the Multi-Model Endpoint

다중 모델 엔드포인트 뒤에서 호스팅하려는 모든 모델에서 예상되는 예측 워크로드에 대해 적절한 인스턴스 유형과 인스턴스 수를 고려해야 합니다. <br/>
개별 모델의 수와 크기도 메모리 요구 사항을 결정합니다.

In [ ]:
try:
    predictor.delete_endpoint(delete_endpoint_config=True)
    print("Deleting previous endpoint...")
    time.sleep(10)
except (NameError, ClientError):
    pass

predictor = mme.deploy(
    initial_instance_count=1,
    instance_type="ml.c5.xlarge",
)
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

### Our endpoint has launched! Let's look at what models are available to the endpoint!

'available'이란 위의 'MultiDataModel', 즉 'model_data_prefix'를 설정할 때 정의한 S3 prefix 아래에 현재 어떤 모델 아티팩트가 저장되어 있는지를 의미합니다.

현재 정의된 S3 prefix 아래에 아티팩트(예: `tar.gz` 파일)가 저장되어 있지 않기 때문에 엔드포인트에는 추론 요청을 처리하는 데 'available' 모델이 없습니다.

아래에서 엔드포인트에서 모델을 'available'하게 만드는 방법을 보여줍니다.

In [ ]:
# No models visible!
list(mme.list_models())

### Dynamically deploying models to the endpoint

`MultiDataModel` 의 `.add_model()` 메소드는 모델 아이팩트를 초기에 저장된 곳 (보통 S3) 으로부터 추론 요청시에 모델 아티팩트를 소싱할 endpoint 로 복사합니다. 

아래에서 보는 것 처럼 요청에 따라 현재 running 중인 endpoint 에 동적으로 모델을 배포할 수 있습니다.

`model_data_source` refers to the location of our model artifact (i.e. where it was deposited on S3 after training completed)

`model_data_source` 는 모델 아티팩트의 위치를 참조합니다. (예를 들어 training 이 완료된 후에 S3 에 저장)

`model_data_path` 는 우리가 위에서 지정한 S3 prefix (`model_data_prefix` : 추론 요청에 대해 모델을 소싱할 endpoint) 에 대한 상대 경로입니다. <br/>
이것이 상대경로이기 때문에 우리는 단순히 추론시에 모델 아티팩트를 호출하기를 원하는 이름만 전달할 수 있습니다. 

> **Note:** 이 예제처럼 훈련 작업 `model.tar.gz` 출력을 직접 사용하려면 훈련 작업이 다음과 같은 결과를 생성하는지 확인해야 합니다.
>
> - `code/` 하위 폴더에 필요한 추론 코드가 이미 포함되어 있으며,
> - (SageMaker PyTorch 컨테이너 v1.6 이상을 사용하는 경우) TorchServe와 호환되도록 패키지 되었습니다.
>
> 이에 대한 참고 사항은 [src/train.py](src/train.py)의 `enable_sm_oneclick_deploy()` 및 `enable_torchserve_multi_model()` 함수를 참조하세요. <br/>
> 또는 실제 학습 후 동일한 단계를 수행하여 원시 학습 작업 결과에서 제공할 준비가 된 새로운 `model.tar.gz`를 생성할 수도 있습니다.

In [ ]:
for name, est in {"ModelA": estimatorA, "ModelB": estimatorB}.items():
    artifact_path = est.latest_training_job.describe()["ModelArtifacts"]["S3ModelArtifacts"]
    # This is copying over the model artifact to the S3 location for the MME.
    mme.add_model(model_data_source=artifact_path, model_data_path=name)

### Our models are ready to invoke!

이제 `MultiDataModel`을 설정할 때 지정한 S3 접두사에 모델 아티팩트가 나열되는 것을 볼 수 있습니다. <br/>
따라서 엔드포인트는 이제 이러한 모델에 대한 추론 요청을 처리할 수 있습니다.

In [ ]:
list(mme.list_models())

## Get predictions from the endpoint

`mme.deploy()`를 재호출하면 우리가 변수 `predictor` 에 저장한 [RealTimePredictor](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/predictor.py#L35)를 반환합니다. 

이제 `predictor`는 호출할 모델을 지정하여 추론을 요청하는데 사용될 수 있습니다.

In [ ]:
dummy_data = get_dummy_request()

start_time = time.time()
predicted_value = predictor.predict(dummy_data, target_model="ModelA")
duration = time.time() - start_time

print(f"Model took {int(duration * 1000):,d} ms")
np.array(predicted_value)[0]

## Updating a model

모델을 업데이트하기 위해 예를 들어 `ModelA-2` 와 같은 새로운 모델로써 추가하고 위와 같은 접근방법을 사용합니다. 

Amazon S3에서 모델 아티팩트를 덮어쓰지 않아야 합니다. <br/>
이전 버전의 모델이 여전히 엔드포인트의 실행 중인 컨테이너 또는 엔드포인트 인스턴스의 스토리지 볼륨에 로드될 수 있기 때문입니다. <br/>
이렇게 하면 추론 호출시 이전 버전을 계속 사용하게 됩니다.

대안으로써 endpoint 를 중지하고 재배포할 수 있습니다. 

## Clean up

endpoint는 배포된 시간에 따라 청구(per the [SageMaker pricing page](https://aws.amazon.com/sagemaker/pricing/))되므로 더 이상 사용하지 않을 때는 삭제해야 합니다. <br/>
여기서 endpoint 구성도 삭제하여 깔끔하게 유지합니다.

In [ ]:
predictor.delete_endpoint(delete_endpoint_config=True)